Name: Sakshi Goyal

Roll Number:102103451

### Objective:

1. Getting the data 
2. Cleaning the data 
3. Organizing the data - organize the cleaned data into a way that is easy to input into other algorithms

### Output :
#### cleaned and organized data in two standard text formats:

1. Corpus - a collection of text
2. Document-Term Matrix - word counts in matrix format

In [35]:
import requests
from bs4 import BeautifulSoup
import re
import pickle
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [36]:
def url_to_transcript(url):
    
    try:
        page = requests.get(url).text
        soup = BeautifulSoup(page, "html.parser")
        para=soup.find_all('p')
        text=[p.text for p in para]
        print(url,'\n')
        print(text)
        return text
    except Exception as e:
        print(f"Error scraping transcript from {url}: {e}")
        return None

urls = ['https://scrapsfromtheloft.com/movies/elemental-2023-transcript/',
        'https://scrapsfromtheloft.com/movies/the-monkey-king-2023-transcript/',
        'https://scrapsfromtheloft.com/movies/despicable-me-3-transcript/',
        'https://scrapsfromtheloft.com/movies/klaus-2019-transcript/',
        'https://scrapsfromtheloft.com/movies/the-boss-baby-family-business-transcript/']
        
TvSeries = ['ELEMENTAL', 'THE MONKEY KING', 'DESPICABLE ME 3', 'KLAUS', 'THE BOSS BABY']

transcripts = [url_to_transcript(u) for u in urls]

!mkdir transcripts

for i, c in enumerate(TvSeries):
    with open("transcripts/" + c + ".txt", "wb") as file:
        pickle.dump(transcripts[i], file)
        
data = {}
for c in TvSeries:
    with open("transcripts/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)


https://scrapsfromtheloft.com/movies/elemental-2023-transcript/ 

['The film journeys alongside an unlikely pair, Ember and Wade, in a city where fire-, water-, land- and air-residents live together. The fiery young woman and the go-with-the-flow guy are about to discover something elemental: how much they actually have in common.', '* * *', '(PEACEFUL NEWAGE MUSIC PLAYING)', '(WAVES LAPPING)', '(WIND WHOOSHES)', '(BRIGHT MUSIC PLAYS)', '(HORN BLOWS)', 'Oh.', '(BELL CLANGS)', '(CHUCKLES)', '(SHIP HORN BLOWS)', '(RUMBLING)', 'Ah!', 'ANNOUNCER: Please keep all limbs and branches inside.', '(BELL CLANGS)', '(BUBBLING)', 'Oh. I believe this is yours.', 'Oh, thanks.', '(SOFT WHIRRING)', '(CHATTERING)', '(BABY CRYING)', '(BRIGHT MUSIC CONTINUES)', '(CHILDREN GIGGLING)', 'MAN: (OVER PA SYSTEM) Welcome to Element City.', 'Please have your documents ready for inspection.', 'IMMIGRATION OFFICIAL: Next.', 'Your names?', '(BOTH SPEAKING FIRISH)', 'Great. And how do we spell that?', '(ROARING, HISS

In [37]:
data.keys()

dict_keys(['ELEMENTAL', 'THE MONKEY KING', 'DESPICABLE ME 3', 'KLAUS', 'THE BOSS BABY'])

## Cleaning The Data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate.


In [38]:
next(iter(data.keys()))

'ELEMENTAL'

In [39]:
next(iter(data.values()))

['The film journeys alongside an unlikely pair, Ember and Wade, in a city where fire-, water-, land- and air-residents live together. The fiery young woman and the go-with-the-flow guy are about to discover something elemental: how much they actually have in common.',
 '* * *',
 '(PEACEFUL NEWAGE MUSIC PLAYING)',
 '(WAVES LAPPING)',
 '(WIND WHOOSHES)',
 '(BRIGHT MUSIC PLAYS)',
 '(HORN BLOWS)',
 'Oh.',
 '(BELL CLANGS)',
 '(CHUCKLES)',
 '(SHIP HORN BLOWS)',
 '(RUMBLING)',
 'Ah!',
 'ANNOUNCER: Please keep all limbs and branches inside.',
 '(BELL CLANGS)',
 '(BUBBLING)',
 'Oh. I believe this is yours.',
 'Oh, thanks.',
 '(SOFT WHIRRING)',
 '(CHATTERING)',
 '(BABY CRYING)',
 '(BRIGHT MUSIC CONTINUES)',
 '(CHILDREN GIGGLING)',
 'MAN: (OVER PA SYSTEM) Welcome to Element City.',
 'Please have your documents ready for inspection.',
 'IMMIGRATION OFFICIAL: Next.',
 'Your names?',
 '(BOTH SPEAKING FIRISH)',
 'Great. And how do we spell that?',
 '(ROARING, HISSING)',
 '(GROWLS)',
 'How about we ju

In [40]:
def combine_text(list_of_text):
   
    combined_text = ' '.join(list_of_text)
    return combined_text

data_combined = {key: [combine_text(value)] for (key, value) in data.items()}


In [41]:
import pandas as pd
pd.set_option('max_colwidth',180)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df = data_df.sort_index()
data_df

,transcript
DESPICABLE ME 3,"In the animated adventure Despicable Me 3, Felonious Gru and Lucy Wilde, married agents of the Anti-Villain League (AVL), are tasked with thwarting the plans of Balthazar Bratt..."
ELEMENTAL,"The film journeys alongside an unlikely pair, Ember and Wade, in a city where fire-, water-, land- and air-residents live together. The fiery young woman and the go-with-the-fl..."
KLAUS,"When Jesper (Jason Schwartzman) distinguishes himself as the postal academy’s worst student, he is stationed on a frozen island above the Arctic Circle, where the feuding local..."
THE BOSS BABY,Tim and his Boss Baby little bro Ted have become adults and drifted away from each other. Tim is now a married dad. Ted is a hedge fund CEO. But a new boss baby with a cutting-...
THE MONKEY KING,"Inspired by an epic Chinese tale, THE MONKEY KING is an action-packed family comedy that follows a charismatic Monkey (Jimmy O. Yang) and his magical fighting Stick on an epic ..."


### Assignment:
1. Perform the following data cleaning on transcripts:
i) Make text all lower case
ii) Remove punctuation
iii) Remove numerical values
iv) Remove common non-sensical text (/n)
v) Tokenize text
vi) Remove stop words

In [42]:
import re
from nltk.tokenize import word_tokenize

def clean_text_round1(text):
   
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\b\w{1,2}\b', '', text)
    tokens = word_tokenize(text)
    stop_words = ['and', 'the', 'is', 'it', 'in', 'of', 'to']  
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)

round1 = lambda x: clean_text_round1(x)


In [43]:
import pandas as pd

data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean

,transcript
DESPICABLE ME 3,animated adventure despicable felonious gru lucy wilde married agents antivillain league avl are tasked with thwarting plans balthazar bratt former child star turned villain af...
ELEMENTAL,film journeys alongside unlikely pair ember wade city where fire water land airresidents live together fiery young woman gowiththeflow guy are about discover something elementa...
KLAUS,when jesper jason schwartzman distinguishes himself postal academys worst student stationed frozen island above arctic circle where feuding locals hardly exchange words let alo...
THE BOSS BABY,tim his boss baby little bro ted have become adults drifted away from each other tim now married dad ted hedge fund ceo but new boss baby with cuttingedge approach cando attitu...
THE MONKEY KING,inspired epic chinese tale monkey king actionpacked family comedy that follows charismatic monkey jimmy yang his magical fighting stick epic quest for victory over demons eccen...


In [44]:

def clean_text_round2(text):
 
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

data_clean = pd.DataFrame(data_clean.transcript.apply(round2))
data_clean

,transcript
DESPICABLE ME 3,animated adventure despicable felonious gru lucy wilde married agents antivillain league avl are tasked with thwarting plans balthazar bratt former child star turned villain af...
ELEMENTAL,film journeys alongside unlikely pair ember wade city where fire water land airresidents live together fiery young woman gowiththeflow guy are about discover something elementa...
KLAUS,when jesper jason schwartzman distinguishes himself postal academys worst student stationed frozen island above arctic circle where feuding locals hardly exchange words let alo...
THE BOSS BABY,tim his boss baby little bro ted have become adults drifted away from each other tim now married dad ted hedge fund ceo but new boss baby with cuttingedge approach cando attitu...
THE MONKEY KING,inspired epic chinese tale monkey king actionpacked family comedy that follows charismatic monkey jimmy yang his magical fighting stick epic quest for victory over demons eccen...


In [45]:
import pandas as pd

data_df['transcript'] = data_df['transcript'].str.replace('\U0001F3B5', '')

data_clean = pd.DataFrame(data_df.transcript.apply(round1))
data_clean



,transcript
DESPICABLE ME 3,animated adventure despicable felonious gru lucy wilde married agents antivillain league avl are tasked with thwarting plans balthazar bratt former child star turned villain af...
ELEMENTAL,film journeys alongside unlikely pair ember wade city where fire water land airresidents live together fiery young woman gowiththeflow guy are about discover something elementa...
KLAUS,when jesper jason schwartzman distinguishes himself postal academys worst student stationed frozen island above arctic circle where feuding locals hardly exchange words let alo...
THE BOSS BABY,tim his boss baby little bro ted have become adults drifted away from each other tim now married dad ted hedge fund ceo but new boss baby with cuttingedge approach cando attitu...
THE MONKEY KING,inspired epic chinese tale monkey king actionpacked family comedy that follows charismatic monkey jimmy yang his magical fighting stick epic quest for victory over demons eccen...


In [53]:
import pandas as pd

data_clean.to_pickle("data_clean.pkl")


In [46]:

for index, row in data_clean.iterrows():
    print(row['transcript'])


animated adventure despicable felonious gru lucy wilde married agents antivillain league avl are tasked with thwarting plans balthazar bratt former child star turned villain after failing secure bratt stolen dumont diamond they are unceremoniously fired from avl their personal lives are upended when most their minions leave gru discovers has twin brother dru who lives wealth wishes become villain like their father gru grapples with his identity reluctance return villainy bratt steals diamond again execute his revenge hollywood brothers eventually team stop bratt leading series misadventures that test their bond culminate dramatic rescue mission finally bring family closer together despite initial tensions conflicts story ends high note with gru lucys reinstatement into avl renewed sense unity among characters setting stage for further adventures illuminat chuckles clears throat illuminat giggles illuminat speaking minionese farts sniffs groans giggles illumination farts groaning year w

## Organizing The Data

### Assignment:
1. Organized data in two standard text formats:
   a) Corpus - corpus is a collection of texts, and they are all put together neatly in a pandas dataframe here.
   b) Document-Term Matrix - word counts in matrix format

In [47]:
full_names = ['ELEMENTAL (2023) ',
              'THE MONKEY KING (2023)',
              'DESPICABLE ME 3 (2017)',
              'KLAUS (2019)',
              'THE BOSS BABY: FAMILY BUSINESS (2021)']

data_df['full_name'] = full_names

corpus_df = pd.DataFrame({'full_name': data_df['full_name'],
                          'transcript': data_clean['transcript']})

corpus_df.to_pickle("corpus.pkl")

corpus_df

data_df.to_pickle("corpus.pkl")


In [48]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

dtm = vectorizer.fit_transform(corpus_df['transcript'])

data_dtm = pd.DataFrame(dtm.toarray(), columns=vectorizer.get_feature_names_out())

data_dtm.index = corpus_df['full_name']

data_dtm


,aah,abandoned,able,about,above,absentminded,absolutely,absurd,abuse,abysmal,...,youyou,yum,yummy,yup,zero,zip,zombielike,zombies,zone,àshfá
full_name,,,,,,,,,,,,,,,,,,,,,
ELEMENTAL (2023),0,0,1,11,1,0,0,0,0,0,...,0,1,0,1,1,0,0,0,0,0
THE MONKEY KING (2023),4,0,2,21,0,0,0,0,0,0,...,0,0,0,3,0,0,0,0,0,10
DESPICABLE ME 3 (2017),17,0,0,22,1,0,2,0,0,1,...,0,0,0,1,0,0,0,0,0,0
KLAUS (2019),8,0,0,38,0,1,2,0,1,0,...,2,0,0,0,0,0,3,2,1,0
THE BOSS BABY: FAMILY BUSINESS (2021),0,2,0,13,1,0,4,1,0,0,...,0,6,2,0,0,1,0,0,0,0


In [49]:

data_dtm.to_pickle("dtm.pkl")

## Additional Assignments:

1. Can you add an additional regular expression to the clean_text_round2 function to further clean the text?
2. Play around with CountVectorizer's parameters. What is ngram_range? What is min_df and max_df?

In [52]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1, 3), min_df=3, max_df=3)

data_cv = cv.fit_transform(data_clean.transcript)

data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names_out())

data_dtm.index = data_clean.index

data_dtm


In [51]:
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
from IPython.display import display, HTML

# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to perform lemmatization on text
def lemmatize_text(text):
    # Tokenize the text into words
    words = nltk.word_tokenize(text)
    # Initialize lists to store original words and their lemmatized forms
    original_words = []
    lemmatized_words = []
    # Apply lemmatization to each word
    for word in words:
        original_words.append(word)
        lemmatized_word = lemmatizer.lemmatize(word)
        lemmatized_words.append(lemmatized_word)
    # Join the lemmatized words back into a single string
    lemmatized_text = ' '.join(lemmatized_words)
    # Return the original words and lemmatized text
    return original_words, lemmatized_text

# Function to highlight changed words
def highlight_changed_words(original_words, lemmatized_words):
    highlighted_words = []
    for original_word, lemmatized_word in zip(original_words, lemmatized_words.split()):
        if original_word != lemmatized_word:
            highlighted_word = f'<span style="color: red; font-weight: bold;">{lemmatized_word}</span>'
        else:
            highlighted_word = lemmatized_word
        highlighted_words.append(highlighted_word)
    return ' '.join(highlighted_words)

# Apply lemmatization to a sample of transcripts in the DataFrame
sample_size = 5  # Adjust this value to change the sample size
sample_transcripts = data_clean['transcript'].sample(sample_size)
sample_data_clean = pd.DataFrame({'transcript': sample_transcripts})
sample_data_clean[['original_words', 'transcript_lemmatized']] = sample_data_clean['transcript'].apply(lemmatize_text).apply(pd.Series)

# Apply highlighting to changed words in the visible part of the DataFrame
sample_data_clean_display = sample_data_clean.copy()
sample_data_clean_display['transcript_lemmatized'] = sample_data_clean_display.apply(lambda row: highlight_changed_words(row['original_words'], row['transcript_lemmatized']), axis=1)

# Display the sample DataFrame with original words and their lemmatized forms, highlighting only changed words and aligning text to justify
html_output = sample_data_clean_display.to_html(escape=False, index=False)
html_output = html_output.replace('<td>', '<td style="text-align: justify;">')

# Display the HTML table
display(HTML(html_output))
